In [ ]:
# TO DO
# remove category titles
# remove duplicates
# remove nb de joueurs, dureee, etc.

In [ ]:
import pandas as pd
import re
from collections import OrderedDict

In [ ]:
content = pd.read_csv('content.csv', encoding='utf-8')
content.shape

In [ ]:
def extract_into_paragraphs(x):
    clean = x.replace('\t','\n').replace('\x0b','\n')
    full = [p.replace('\xa0',' ') for p in clean.split('\n') if p != '']
    return full

In [ ]:
content['content_clean'] = content['content'].apply(extract_into_paragraphs)

In [ ]:
section_titles = ['.{0,4}?[é|e]chauffement.{0,10}?','.{0,4}?exercices.{0,10}?', '.{0,4}?matchs?.{0,10}?',
                  '.{0,4}?scè|enes.{0,10}?','.{0,4}?introduction.{0,10}?','.{0,4}?th[e|è]mes.{0,10}?','.{0,14}?Jeux.{0,10}?',
                  '.{0,4}?autres: ']
sections_titles_clean = {section_titles[0]:'Echauffement',
           section_titles[1]:'Exercices',
           section_titles[2]:'Matches',
           section_titles[3]:'Matches',
           section_titles[4]:'',
           section_titles[5]:'Thèmes',
           section_titles[6]:'Exercices',
           section_titles[7]:'Matches'}

In [ ]:
content_lengths = [len(x) for x in content['content_clean']]    
content['new_index'] = content.apply(lambda x: sum(content_lengths[0: x['index']]), axis=1)    

In [ ]:
def build_section_indexes(x):
    sections = OrderedDict()
    for index, line in enumerate(x['content_clean']):
        for title in section_titles:
            if re.match(title, line.lower()):
                i = x['new_index'] + index
                sections[i] = sections_titles_clean[title]
    return sections

#build_section_indexes(content.iloc[2])
content['sections'] = content.apply(build_section_indexes, axis=1)    

In [ ]:
content_long = content.explode('content_clean')

In [ ]:
content_long.reset_index(inplace=True)

In [ ]:
content_long.shape

In [ ]:
def assign_categories(x):
    for key in x['sections'].keys():
        if x.name==key:
            return x.name
content_long['categories'] = content_long.apply(assign_categories, axis=1)

In [ ]:
content_long = content_long[content_long['content_clean']!=' ']
content_long.shape

In [ ]:
content_long['categories_index'] = content_long.groupby('index')['categories'].apply(lambda x: x.fillna(method='ffill'))
content_long['categories_index'].fillna(0, inplace=True)
content_long['categories'] = content_long.apply(lambda x: x['sections'][x['categories_index']] if x['categories_index']>0 else '', axis=1)

In [ ]:
content_long[['title','content_clean','categories']].to_csv('content_long.csv', encoding='utf-8', index=False)